Loading the model in 

In [65]:
import torch
import pandas as pd
from torch import nn 
import joblib

# from common_utils import MLPBuilder
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from sklearn.model_selection import train_test_split


In [66]:
class MLPBuilder(nn.Module):

    def __init__(self, no_features, layers, no_labels = 64):
        super().__init__()
        layer_list = []
        
        # Input layer
        layer_list.append(nn.Linear(no_features, layers[0]))
        layer_list.append(nn.ReLU())
        layer_list.append(nn.Dropout(p=0.2))

        # Hidden layers
        for i in range(len(layers) - 1):
            layer_list.append(nn.Linear(layers[i], layers[i+1]))
            layer_list.append(nn.ReLU())
            layer_list.append(nn.Dropout(p=0.2))

        # Output layer, 2 outputs
        layer_list.append(nn.Linear(layers[-1], 2))
 
        # Define the MLP stack as a sequential model
        self.mlp_stack = nn.Sequential(*layer_list)

        self._initialize_weights() 
        
    def forward(self, x):
        logits = self.mlp_stack(x)
        return logits
    
    def _initialize_weights(self):
        for layer in self.mlp_stack:
            if isinstance(layer, nn.Linear):
                # Use Kaiming initialization for ReLU activations
                nn.init.kaiming_uniform_(layer.weight, nonlinearity='relu')
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)

In [67]:
# we trained the model with 8 features

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPBuilder(no_features=8, layers=[256, 512, 512, 128])
model.load_state_dict(torch.load('models/Regression_based/DNN_model_[256, 512, 512, 128].pth', map_location=device))
model.to(device)
model.eval()  

C:\Users\randa\AppData\Local\Temp\ipykernel_18608\2038283039.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models/Regression_based/DN

MLPBuilder(
  (mlp_stack): Sequential(
    (0): Linear(in_features=8, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=512, out_features=128, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.2, inplace=False)
    (12): Linear(in_features=128, out_features=2, bias=True)
  )
)

## Loading test data in 

In [68]:


train_file_path = './Data/25-02-10/cleaned_df.csv'
test_file_path = './Data/25-02-04/combined_data.csv'

model_path = './models/Regression_based'
scaler_X = joblib.load(model_path +'/scaler_X.pkl')
scaler_Y = joblib.load(model_path +'/scaler_Y.pkl')

train_cleaned_df = pd.read_csv(train_file_path)

In [69]:
train_cleaned_df

,Tx_0 RSSI,Tx_1 RSSI,Tx_2 RSSI,Tx_3 RSSI,Tx_4 RSSI,Tx_5 RSSI,Tx_6 RSSI,Tx_7 RSSI,X_Coord,Y_Coord
0,-61,-74,-79,127,-81,-78,127,-84,0,0
1,-61,-73,-78,-71,127,-78,127,127,0,0
2,-62,-75,-78,-72,-79,127,-84,127,0,0
3,-61,-73,-80,-71,-83,-78,127,127,0,0
4,-61,-75,-78,-71,-84,-78,-84,-86,0,0
...,...,...,...,...,...,...,...,...,...,...
6395,-79,-69,-81,-74,-70,-61,-78,-68,6,0
6396,-77,-69,-77,-71,-70,-61,-79,-67,6,0
6397,-77,-69,-81,-73,-70,-61,-79,-66,6,0
6398,-80,-69,-77,-72,-71,-61,-76,-67,6,0


In [70]:
# batt_columns = [col for col in cleaned_df.columns if 'Batt' in col]
# time_columns = [col for col in cleaned_df.columns if 'Time' in col]
# columns_to_drop = batt_columns+time_columns
# RSSI_columns = cleaned_df.drop(columns=columns_to_drop)


#Train the model without NAN values
# cleaned_df = RSSI_columns.dropna()



# Evaluation on original dataset

In [71]:
X_train = train_cleaned_df[['Tx_0 RSSI', 'Tx_1 RSSI', 'Tx_2 RSSI', 'Tx_3 RSSI', 'Tx_4 RSSI', 'Tx_5 RSSI', 'Tx_6 RSSI', 'Tx_7 RSSI']]
Y_train = train_cleaned_df[['X_Coord', 'Y_Coord']]

X_train_scaled = scaler_X.transform(X_train)
Y_train_scaled = scaler_Y.transform(Y_train)

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train_scaled, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)


In [72]:
def evaluate_model(model, dataloader):
    model.eval()  # Set to evaluation mode
    total_displacement_error = 0.0
    total_mse_loss = 0.0
    total_samples = 0
    criterion = nn.MSELoss()  # MSE loss function

    with torch.no_grad():  
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)  # Move to GPU if available
            
            preds = model(inputs)  # Model outputs predicted X and Y coordinates

            # Compute MSE loss
            mse_loss = criterion(preds, targets)

            # Compute displacement error (Euclidean distance)
            displacement = torch.sqrt(torch.sum((preds - targets) ** 2, dim=1))
            
            # Accumulate total error
            total_mse_loss += mse_loss.item() * targets.size(0)  # Weighted sum for averaging
            total_displacement_error += displacement.sum().item()
            total_samples += targets.size(0)

    # Compute mean values
    avg_loss = total_mse_loss / total_samples
    avg_displacement_error = total_displacement_error / total_samples
    
    return avg_loss, avg_displacement_error


In [73]:
avg_loss, displacement_loss = evaluate_model(model, train_loader)

# Print results
print(f"Test MSE Loss: {avg_loss:.6f}")
print(f"Test Displacement Loss: {displacement_loss:.6f}") 

Test MSE Loss: 0.025189
Test Displacement Loss: 0.171381


# Implementing transfer learning in test environment

In [74]:
test_cleaned_df = pd.read_csv(test_file_path)

In [75]:
X_test = test_cleaned_df[['Tx_0 RSSI', 'Tx_1 RSSI', 'Tx_2 RSSI', 'Tx_3 RSSI', 'Tx_4 RSSI', 'Tx_5 RSSI', 'Tx_6 RSSI', 'Tx_7 RSSI']]
Y_test = test_cleaned_df[['X_Coord', 'Y_Coord']]

X_test, X_val, Y_test, Y_val  = train_test_split(X_test, Y_test, test_size=0.2, random_state=42)

X_val_scaled = scaler_X.transform(X_val)
Y_val_scaled = scaler_Y.transform(Y_val)
X_test_scaled = scaler_X.transform(X_test)
Y_test_scaled = scaler_Y.transform(Y_test)

# Convert to PyTorch tensors
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
Y_val_tensor = torch.tensor(Y_val_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test_scaled, dtype=torch.float32)

val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

# Create DataLoader for the test set
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [76]:
avg_loss, displacement_loss = evaluate_model(model, test_loader)

# Print results
print(f"Test MSE Loss: {avg_loss:.6f}")
print(f"Test Displacement Loss: {displacement_loss:.6f}") 

Test MSE Loss: 0.108426
Test Displacement Loss: 0.418017


In [77]:
for param in model.mlp_stack[:-1].parameters():  # Freeze all except last layer
    param.requires_grad = False

num_features = model.mlp_stack[-1].in_features
model.mlp_stack[-1] = nn.Linear(num_features, 2)


# Move model to device
model.to(device)

# Verify layer status
for name, param in model.named_parameters():
    print(name, param.requires_grad)


criterion = nn.MSELoss()
optimizer = optim.Adam(model.mlp_stack[-1].parameters(), lr=0.001, weight_decay=1e-4)  # 



mlp_stack.0.weight False
mlp_stack.0.bias False
mlp_stack.3.weight False
mlp_stack.3.bias False
mlp_stack.6.weight False
mlp_stack.6.bias False
mlp_stack.9.weight False
mlp_stack.9.bias False
mlp_stack.12.weight True
mlp_stack.12.bias True


In [82]:
num_epochs = 100
patience = 5  
best_loss = float('inf')
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_samples = 0

    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()  # Zero gradients
        preds = model(inputs)  # Forward pass
        loss = criterion(preds, targets)  # Compute loss

        loss.backward()  # Backpropagation
        optimizer.step()  # Update last layer weights

        total_loss += loss.item() * targets.size(0)  
        total_samples += targets.size(0)

    avg_loss = total_loss / total_samples  
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.6f}")

    # Early stopping logic
    if avg_loss < best_loss:
        best_loss = avg_loss
        epochs_no_improve = 0  # Reset counter if loss improves
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break  # Stop training if no improvement for `patience` epochs

print("Transfer learning complete.")
# torch.save(model.state_dict(), "models/Regression_based/DNN_model_[256, 512, 512, 128]_finetuned.pth")


Epoch [1/100], Loss: 0.086393
Epoch [2/100], Loss: 0.087316
Epoch [3/100], Loss: 0.087216
Epoch [4/100], Loss: 0.087418
Epoch [5/100], Loss: 0.087143
Epoch [6/100], Loss: 0.087700
Early stopping triggered after 6 epochs.
Transfer learning complete.


In [81]:
avg_loss, displacement_loss = evaluate_model(model, val_loader)

# Print results
print(f"Test MSE Loss: {avg_loss:.6f}")
print(f"Test Displacement Loss: {displacement_loss:.6f}") 

Test MSE Loss: 0.083832
Test Displacement Loss: 0.373712
